# OpenStreetMap Dataset
This OpenStreetMap Dataset notebook was created for the final project of the _Data Wrangling with MongoDB_ Udacity course. 
Specifically, this project is the _OpenStreetMap Data_ project. 

This notebook forms an analysis of an OpenStreetMap dataset. OpenStreetMap is a collaborative project to create a free editable map of the world. The data generated by the OpenStreetMap project is considered its primary output, rather than the map itself. The data can be used in a variety of applications in place of propriety datasources. Data can be downloaded from the OSM website in different formats, including XML. This dataset was obtained from the [Metro Extracts section](https://mapzen.com/data/metro-extracts/metro/sydney_australia/) of the OpenStreetMap website. It is the Sydney, Australia metro extract.

This dataset can be investigated and wrangled with the purpose of preparing it for entering into a MongoDB NoSQL collection.

The _Data Wrangling with MongoDB_ course serves as one of the courses leading to the Udacity Data Analyst Nanodegree.

## Investigate the Dataset
### Elements
Find what elements are present in the XML data file and how many of each type. We use iterative parsing to process the map file to avoid loading the whole file into memory.

In [3]:
import xml.etree.cElementTree as ET

filename = "sydney_australia.osm"
elements = {}
for event, elem in ET.iterparse(filename):
    tag = elem.tag
    if tag in elements:
        elements[tag] += 1
    else:
        elements[tag] = 1
elements

{'bounds': 1,
 'member': 71110,
 'nd': 1755778,
 'node': 1461112,
 'osm': 1,
 'relation': 5179,
 'tag': 839841,
 'way': 200653}

According to the documentation, OpenStreetMap uses a topological data structure, with mapping points represented by three core elements:
* __Nodes__ are points with a geographic position
* __Ways__ are ordered lists of nodes
* __Relations__ are ordered lists of nodes

In addition, __tags__ are sub-elements which are used to store metadata about their respective map objects. Tag data is stored in key-value pairs.

### Tags
Investigate what tags are present in the XML data.

In [4]:
tags = {}
for event, element in ET.iterparse(filename):
    if element.tag == "tag":
        key = element.attrib['k']
        if key in tags:
            tags[key] += 1
        else:
            tags[key] = 1
print("Number of tag types found: %d" % len(tags))
sorted_tags = sorted(tags.items(), reverse=True, key=lambda (k,v): (v,k))

print "Printing the first 20 results:"
for i in range(0, 20):
    print sorted_tags[i]

Number of tag types found: 1859
Printing the first 20 results:
('highway', 146486)
('name', 100696)
('source', 97995)
('building', 39895)
('surface', 25754)
('maxspeed', 24982)
('oneway', 20365)
('addr:street', 18864)
('amenity', 18144)
('source:name', 13595)
('noname', 12472)
('created_by', 12330)
('foot', 12009)
('leisure', 11852)
('addr:housenumber', 10191)
('bicycle', 10186)
('addr:postcode', 8056)
('service', 8024)
('lanes', 7813)
('ref', 7248)


The dataset contains 1859 types of tags. Of particular note, is the fact that address information is spread over multiple tags. Each tag name being prefixed with "addr:". This data may need to be wrangled before being entered into the MongoDB database.

### Address tags
Investigate the various types of tags used for address data.

In [5]:
addrs = {}
for key in tags:
    if key[:5] == "addr:":
        addrs[key] = tags[key]

print("Number of address tag types found: %d" % len(addrs))
sorted(addrs.items(), reverse=True, key=lambda (k,v): (v,k))

Number of address tag types found: 29


[('addr:street', 18864),
 ('addr:housenumber', 10191),
 ('addr:postcode', 8056),
 ('addr:city', 6517),
 ('addr:country', 2177),
 ('addr:state', 1735),
 ('addr:suburb', 1556),
 ('addr:housename', 320),
 ('addr:province', 273),
 ('addr:interpolation', 178),
 ('addr:place', 109),
 ('addr:unit', 39),
 ('addr:floor', 13),
 ('addr:source:housenumber', 7),
 ('addr:shop', 5),
 ('addr:suite', 3),
 ('addr:street:source', 2),
 ('addr:level', 2),
 ('addr:flats', 2),
 ('addr:city_1', 2),
 ('addr:street_1', 1),
 ('addr:room', 1),
 ('addr:number', 1),
 ('addr:lot', 1),
 ('addr:inclusion', 1),
 ('addr:housenumber_1', 1),
 ('addr:housenumber:source', 1),
 ('addr:full', 1),
 ('addr:building', 1)]

29 types of address tags are present in the dataset.

## Investigate Potential Problems
Before the data is processed and entered into the database, we check the attributes
of each tag and see if there are any potential problems. We use regular expressions to search for certain patterns in the tags that could be problematic and need cleaning. We investigate how many other tags use colons to create sub-tags and how many tags contain potential problem characters.


In [6]:
import re

lower_case = re.compile(r'^([a-z]|_)*$')
lower_case_with_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problem_chars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

keys = {"lower_case": 0, 
        "lower_case_with_colon": 0, 
        "problem_chars": 0, 
        "other": 0}
problem_tags = {}
colon_tags = {}
other_tags = {}

for event, element in ET.iterparse(filename):
    if element.tag == "tag":
        if 'k' in element.attrib:
            k = element.attrib['k']
            if re.search(lower_case, k):
                keys["lower_case"] += 1
            elif re.search(lower_case_with_colon, k):
                keys["lower_case_with_colon"] += 1
                
                if k in colon_tags:
                    colon_tags[k] += 1
                else:
                    colon_tags[k] = 1
            elif re.search(problem_chars, k):
                keys["problem_chars"] += 1
                
                if k in problem_tags:
                    problem_tags[k] += 1
                else:
                    problem_tags[k] = 1
            else:
                keys["other"] += 1
                
                if k in other_tags:
                    other_tags[k] += 1
                else:
                    other_tags[k] = 1

keys

{'lower_case': 731819,
 'lower_case_with_colon': 99458,
 'other': 8557,
 'problem_chars': 7}

There are only 7 tags with characters that could be problematic.

In [7]:
problem_tags

{'Hours of Operation': 2,
 'Old Fox Name': 1,
 'Payments Accepted': 1,
 'acma.gov.au:site_id': 1,
 'entrance:loading dock': 1,
 'old Fox Name': 1}

None of these tags contain important information so tags with problematic characters in the tag name can be safely ignored. We need to look at tags which contain colons in their names.

In [8]:
sorted_tags = sorted(colon_tags.items(), reverse=True, key=lambda (k,v): (v,k))

print "Printing the first 20 results:"
for i in range(0, 20):
    print sorted_tags[i]

Printing the first 20 results:
('addr:street', 18864)
('source:name', 13595)
('addr:housenumber', 10191)
('addr:postcode', 8056)
('addr:city', 6517)
('is_in:suburb', 4093)
('source:location', 3910)
('source:maxspeed', 3220)
('addr:country', 2177)
('source:geometry', 2152)
('name:en', 2117)
('building:levels', 2113)
('source:date', 1873)
('addr:state', 1735)
('ref:start_date', 1622)
('addr:suburb', 1556)
('nswlpi:cadid', 731)
('source:sport', 594)
('ref:nswgnb', 518)
('place:nswgnb', 518)


Principally of note are the tags with address information. Another tag that would be valuable is the "is_in:suburb" tag. Some prominent locations don't have address information recorded but have an "is_in:suburb" tag. All other tags with colons will be ignored.

We need to investigate the other types of tag names that will be encountered.

In [9]:
sorted_tags = sorted(other_tags.items(), reverse=True, key=lambda (k,v): (v,k))

print "Printing the first 20 results:"
for i in range(0, 20):
    print sorted_tags[i]

Printing the first 20 results:
('source:name:date', 4418)
('source:sport:date', 310)
('building:roof:shape', 196)
('parking:lane:both:parallel', 193)
('Origin_Z', 165)
('Origin_Y', 165)
('Origin_X', 165)
('Item_class', 165)
('turn:lanes:forward', 160)
('turn:lanes:backward', 156)
('Item_ID', 145)
('parking:lane:both', 125)
('Floor_Coun', 116)
('Room_Count', 67)
('parking:lane:left', 63)
('parking:lane:both:width', 59)
('shoulder:left:width', 57)
('shoulder:right:width', 54)
('parking:lane:left:parallel', 53)
('cycleway:lane:width', 52)


We can see that some tags have multiple colons in their name. We don't need this information so we will ignore all tags with more than one colon in the name.

## Inconsistencies in the data
### Street Types
Any dataset which contains address information will have inconsistant naming conventions, especially for street types. For example, a road could be called 'Road', 'Rd' or 'Rd.' We will make a list of all street types and look for inconsistencies.

In [10]:
from collections import defaultdict

# Use a regular expression to extract the street type from the "addr:street" tag.
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

street_types = defaultdict(int)
for event, elem in ET.iterparse(filename):
    if is_street_name(elem):
        m = street_type_re.search(elem.attrib['v'])
        if m:
            street_type = m.group()
            street_types[street_type] += 1
        
sorted_tags = sorted(street_types.items(), reverse=True, key=lambda (k,v): (v,k))

print "Printing the first 20 results:"
for i in range(0, 20):
    print sorted_tags[i]

Printing the first 20 results:
('Street', 8783)
('Road', 4525)
('Avenue', 2997)
('Parade', 469)
('Drive', 396)
('Place', 319)
('Crescent', 273)
('Highway', 245)
('Circuit', 169)
('Lane', 83)
('Wolli', 74)
('Court', 52)
('Way', 46)
('Esplanade', 36)
('Close', 33)
('Boulevard', 29)
('Broadway', 25)
('Berith', 25)
('West', 20)
('St', 20)


The list of street types shows some inconsistency in the use of Street, Avenue, Road and Place. The abbreviations 'St', 'St.', 'street', 'st' and even 'Streett' are used for Street. Steps will be taken in the data wrangling phase to remove these abbreviations and make the naming of ways more consistent.

### Apartment numbering
Another inconsistency found in Australian addresses is the way that apartment numbers are represented. For example, apartment 3 at housenumber 42 can be represented as 'Unit 3, 42', '3/42', '3-42', etc. We can see this when we investigate housenumber tags that have a value which contains non-alphanumeric characters.

In [12]:
# Searching for problem chars in the attribute value, v, of tags.
def is_housenumber(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:housenumber")

problemhousenumbers = set()
for event, elem in ET.iterparse(filename):
    if is_housenumber(elem):
        v = elem.attrib['v']
        if problem_chars.search(v):
            problemhousenumbers.add(v)
        
#problemhousenumbers
print "Printing the first 50 results:"
for i, phn in enumerate(problemhousenumbers):
    if i < 50:
        print phn
    else:
        break

Printing the first 50 results:
412-414 George St
9/10
B01/262
Shop P48
Building 15 177-219
7/40
suite 5/68
3/156-158
69/1
Shop 5, 39-53
12/15-13
Shop 2-3, 348
1/63-71
26/29-13
Shop 6/11-13
1/417
3/194-196
518/50
Shop 21, 230
W2C1/75-85
Shop 202, 24-32
2/52
2/400
63/919
2/19-21
1/54
370, Suite 30
11/19-21
54, Suite 202
C1, 85-113
1 / 284
7A/2
29, Suite 123
416,418
Shop 70/46
5/151
210/13
Shop 3c
NEP 10
Shop 14
Gate A, 153-233
2 Level 13
3/1
1/234
1/235
Unit 2 / 1
28/12-14
Shop 4, 14
5/654
10/17


We can see that most apartments are represented in the '3/42' format. Sometimes these numbers are separated by commas, but commas are also used in some cases to form lists. This makes it difficult to extract apartment numbers by looking for commas. The best way to create the most uniformity is to split all housenumbers with a slash, '/', and create a field called `unitnumber`. We can also change the name of the `housenumber` field to `streetnumber` which is a more common terminology in Australia.

## Data Wrangling
We can now process the XML dataset to produce a cut-down JSON document with only the information we want. As we process each element, we will shape the data to take the form we would like it to have in the database. 

As we process the data, we will ignore all elements whose attributes contain problem characters or more than one colon. We will use a dictionary to form a mapping between street type abbreviations and the full name and we will split `housenumber` to form `unitnumber` and `streetnumber`.

In [13]:
import codecs
import json

mapping = { "St": "Street",
            "St.": "Street",
            "street" : "Street",
            "Streett" : "Street",
            "st" : "Street",
            "Ave": "Avenue",
            "Av" : "Avenue",
            "Rd.": "Road",
            "Rd" : "Road"
            }

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z_]*):([a-z_]*)$')
lower_slash = re.compile(r'^([a-z0-9_-]*)\/([a-z0-9_-]*)$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
double_colon = re.compile(r'^([a-z]|_)*:([a-z_]*):([a-z]|_)*$')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def update_street_name(name, mapping):

    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(street_type, mapping[street_type], name)

    return name


def shape_element(element):
    
    node = {}
    if element.tag == "node" or element.tag == "way" :
        for tag in element.iter("tag"):
            k = tag.attrib["k"]
            v = tag.attrib["v"]
            if not (re.search(problemchars, k) or
                    re.search(double_colon, k)):
                match = re.search(lower_colon, k)
                
                # Check if tag attribute has a colon amongst the lower case text
                if match:
                    dict_name = match.group(1)
                    key_name = match.group(2)
                    
                    if dict_name == "addr":
                        if "address" not in node:
                            node["address"] = {}
                        if key_name == "street":
                            v = update_street_name(v, mapping)
                        if key_name == "housenumber":
                            match_slash = re.search(lower_slash, v)
                            
                            if match_slash:
                                node["address"]["unitnumber"] = match_slash.group(1)
                                node["address"]["streetnumber"] = match_slash.group(2)
                            else:
                                node["address"]["streetnumber"] = v
                        else:
                            node["address"][key_name] = v
                            
                    elif (dict_name == "is_in") and (key_name == "suburb"):
                        node["suburb"] = v
                
                # else if only lower case text then copy as is.
                elif re.search(lower, k):
                    node[k] = v

        # Check if any valid tags were found for this element. If not, then lets ignore it.
        if node:
            node["type"] = element.tag
            node["created"] = {}
            node["pos"] = [0.0, 0.0]
            for attribute in element.attrib:
                if attribute in CREATED:
                    node["created"][attribute] = element.attrib[attribute]
                elif attribute == "lat":
                    try:
                        node["pos"][0] = float(element.attrib["lat"])
                    except:
                        node["pos"][0] = 0.0
                elif attribute == "lon":
                    try:
                        node["pos"][1] = float(element.attrib["lon"])
                    except:
                        node["pos"][1] = 0.0
                else:
                    node[attribute] = element.attrib[attribute]

            return node
        else:
            return None
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)

    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

data = process_map(filename, True)

## Importing the JSON document into MongoDB
In MongoDB, we can create an `openstreetdata` database and then create a collection for OSM Metro dataset imported into it. In this case, we can use the following commands to create a `Sydney` collection.

`$ mongo
    > use openstreetdata
    > db.createCollection("Sydney")`

Our JSON document can then be imported into the collection using mongoimport as follows:

`$ mongoimport -d openstreetdata -c Sydney --type json --file sydney_australia.osm.json`

## Querying The Database
With the dataset now imported into MongoDB as a document collection, we can simply query the database to discover any location information about Sydney, Australia. For instance, we can query the dataset for all the types of amenities listed.

In [14]:
from pymongo import MongoClient
client = MongoClient('localhost:27017')
db = client['openstreetdata']

In [15]:
# Build the aggregation pipeline
pipeline = [
            {"$match":{"amenity": {"$exists": 1}}},
            {"$group":{"_id":"$amenity",
                       "count":{"$sum":1}}},
            {"$sort": {"count": -1}},
           ]

result = [doc for doc in db.Sydney.aggregate(pipeline)]

result_length = len(result)
print "Number of types of amenities: %d" % result_length
print "Printing the first 20 results:"
for i in range(0,20):
    print result[i]

Number of types of amenities: 163
Printing the first 20 results:
{u'count': 4106, u'_id': u'parking'}
{u'count': 1414, u'_id': u'bench'}
{u'count': 1069, u'_id': u'school'}
{u'count': 1067, u'_id': u'restaurant'}
{u'count': 899, u'_id': u'cafe'}
{u'count': 828, u'_id': u'toilets'}
{u'count': 782, u'_id': u'drinking_water'}
{u'count': 675, u'_id': u'fast_food'}
{u'count': 627, u'_id': u'place_of_worship'}
{u'count': 541, u'_id': u'bicycle_parking'}
{u'count': 481, u'_id': u'fuel'}
{u'count': 479, u'_id': u'shelter'}
{u'count': 452, u'_id': u'pub'}
{u'count': 421, u'_id': u'post_box'}
{u'count': 358, u'_id': u'post_office'}
{u'count': 322, u'_id': u'waste_basket'}
{u'count': 278, u'_id': u'bbq'}
{u'count': 252, u'_id': u'bank'}
{u'count': 237, u'_id': u'telephone'}
{u'count': 209, u'_id': u'pharmacy'}


The dataset contains 163 types of amenities, the most prevalent of which being parking locations. We could now query the database to find all restaurants and cafes listed.

In [16]:
# Build the aggregation pipeline
pipeline = [
            {"$match": {"amenity": "restaurant",
                        "amenity": "cafe"}},
            {"$project": {"Name": "$name",
                          "Style": "$amenity",
                          "Lat/Lon": "$pos"
                         } 
            }
           ]

result = [doc for doc in db.Sydney.aggregate(pipeline)]

result_length = len(result)
print "Number of restaurants & cafes: %d" % result_length
#result
print "Printing the first 10 results:"
for i in range(0,10):
    #print "result[%s]" % str(i)
    print result[i]

Number of restaurants & cafes: 899
Printing the first 10 results:
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca5f6a'), u'Lat/Lon': [-33.9505592, 151.1634195]}
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca6009'), u'Name': u'Centennial Park Cafe', u'Lat/Lon': [-33.8946861, 151.2333301]}
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca6046'), u'Name': u'Bytes @ ATP', u'Lat/Lon': [-33.8949115, 151.1954059]}
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca6047'), u'Name': u'Saint-Germaine Patisserie', u'Lat/Lon': [-33.894917, 151.1980919]}
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca604f'), u'Name': u"Bobby's Cafe", u'Lat/Lon': [-33.8911183, 151.1958773]}
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca6083'), u'Name': u'Pavilion Caf\xe9', u'Lat/Lon': [-33.948136, 151.2564277]}
{u'Style': u'cafe', u'_id': ObjectId('5a12762405ab824ba3ca616e'), u'Name': u'Sea Salt Cafe Kiosk', u'Lat/Lon': [-33.9143366, 151.2667604]}
{u'Styl

Many locations contain an address or suburb, but every one at least contains a pair of latitude/longitude coordinates. These coordinates could be used to calculate distances between points or to render the locations on a map. 

We could also query the database to find cinemas with wheelchair access.

In [17]:
# Build the aggregation pipeline
pipeline = [
            {"$match":{"amenity": "cinema",
                       "wheelchair": {"$eq": 'yes'}}},
            {"$project": {"name": "$name",
                          "location": "$address.city",
                          "suburb": "$suburb",
                          "lat/lon": "$pos"
                         } 
            }
           ]

result = [doc for doc in db.Sydney.aggregate(pipeline)]

result_length = len(result)
print "Number of cinemas with wheelchair access: %d" % result_length
result

Number of cinemas with wheelchair access: 7


[{u'_id': ObjectId('5a12762405ab824ba3ca6359'),
  u'lat/lon': [-33.8598082, 151.213016],
  u'name': u'Dendy'},
 {u'_id': ObjectId('5a12762505ab824ba3caa9b2'),
  u'lat/lon': [-33.7979086, 151.1820698]},
 {u'_id': ObjectId('5a12762505ab824ba3cab17b'),
  u'lat/lon': [-33.9188443, 150.9233527],
  u'name': u'Event Cinemas'},
 {u'_id': ObjectId('5a12762505ab824ba3cac2f0'),
  u'lat/lon': [-33.8331572, 151.0860894],
  u'name': u'Reading Cinema'},
 {u'_id': ObjectId('5a12762505ab824ba3cad288'),
  u'lat/lon': [-33.8751736, 151.2060563],
  u'location': u'Sydney',
  u'name': u'Event'},
 {u'_id': ObjectId('5a12762505ab824ba3cb096d'),
  u'lat/lon': [-33.8960846, 151.1805118],
  u'name': u'Dendy Newtown'},
 {u'_id': ObjectId('5a12762505ab824ba3cb0a7f'),
  u'lat/lon': [-33.7967566, 151.1830424]}]

## Conclusion
This notebook has demonstrated an application of data wrangling where free mapping data accessible from the OpenStreetMap project in the form of XML data, can be wrangled and processed using standard Python libraries and then loaded into a MongoDB database as a collection of JSON documents. This data can now be used as a backend for a mapping application without needing to use a commercial, propriety mapping service such as Google Maps.